In [166]:
import sys
sys.path.append('../')

In [167]:
import pandas
import numpy
import numpy.linalg

import lib.utils as utils

In [168]:
timus = pandas.read_csv('../data/timus.csv', index_col=0)

/home/ec2-user/mount_point/tfnewenv/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [169]:
filtered_timus = utils.filter_timus_by_unique_accepted(timus, lambda cnt: cnt > 100)

In [170]:
def suggest(timus,
            top_count,
            enable_negative,
            k=200,
           ):
    solved_matrix, authors_encoder, problems_encoder = utils.build_solved_matrix(timus=timus, enable_negative=enable_negative)
    U, S, VT = numpy.linalg.svd(solved_matrix, full_matrices=False)
    A = (U * S)[:,:k]
    B = VT[:k,:]
    C = A.dot(B)
    suggestions = dict()
    for author_id in range(authors_encoder.get_set_size()):
        user_suggestions = []
        for problem_id in range(problems_encoder.get_set_size()):
            if solved_matrix[author_id][problem_id] == 1:
                continue
            real_problem_id = problems_encoder.decode(problem_id)
            user_suggestions.append(
                utils.SingleSuggest(
                    problem_index=real_problem_id,
                    score=C[author_id][problem_id]
                )
            )
        real_author_id = authors_encoder.decode(author_id)
        suggestions[real_author_id] = sorted(user_suggestions, key=lambda s: s.score, reverse=True)[:top_count]
    return suggestions

In [172]:
s = suggest(filtered_timus, top_count=5, enable_negative=False)

In [173]:
s[99162]

[Suggest[problem_index = 1531, score = 0.8464132171213687],
 Suggest[problem_index = 1705, score = 0.7664520486901404],
 Suggest[problem_index = 1802, score = 0.7421519173429595],
 Suggest[problem_index = 1946, score = 0.6874396404739825],
 Suggest[problem_index = 1420, score = 0.6748694599989641]]

In [174]:
s[97440]

[Suggest[problem_index = 1674, score = 0.8562424143195675],
 Suggest[problem_index = 1621, score = 0.8167514402580258],
 Suggest[problem_index = 1853, score = 0.7790755056906884],
 Suggest[problem_index = 1815, score = 0.7690374997186413],
 Suggest[problem_index = 1594, score = 0.7592231214094446]]

In [163]:
svd_based_scores = utils.get_algorithm_scores(
    timus=filtered_timus,
    suggest_for_timus=lambda t: suggest(timus=t, top_count=5, enable_negative=True),
)

In [164]:
svd_based_scores

array([0.09347443, 0.09370988, 0.04859611, 0.06557377, 0.06360424,
       0.06374172, 0.04435798, 0.06997743, 0.06642336, 0.07320644,
       0.06546763, 0.09397944])

In [165]:
pandas.Series(svd_based_scores).describe()

count    12.000000
mean      0.070176
std       0.016367
min       0.044358
25%       0.063707
50%       0.065999
75%       0.078273
max       0.093979
dtype: float64

In [7]:
svd_based_scores

array([0.08289242, 0.09884467, 0.05939525, 0.06557377, 0.07773852,
       0.06870861, 0.05992218, 0.07298721, 0.0729927 , 0.08052709,
       0.07697842, 0.10279001])

In [8]:
pandas.Series(svd_based_scores).describe()

count    12.000000
mean      0.076613
std       0.013555
min       0.059395
25%       0.067925
50%       0.074986
75%       0.081118
max       0.102790
dtype: float64